# **Data Analysis in Python for R Users — Python**

# 0. Environment Setup

#### 1. Loading in libraries

In [ ]:
# Python uses import() for libraries 
import pandas as pd

# We'll import {numpy} for numerical data handling
import numpy as np 

# Load the {pyjanitor} package for cleaning column names
# NB: The import call here is different from the actual library name 
import janitor

# And load {siuba{ for dplyr-styled data manipulation
from siuba import *

# Load the {geopandas} library for handling spatial data and geocoding
import geopandas as gp
from geopandas.tools import geocode

# Load {plotnine} for ggplot2-styled data visualization
from plotnine import * 

# And the fomula api from {statsmodels}
import statsmodels.formula.api as smf

#### 2. Defining custom functions

In [ ]:
# Define a custom function affiche() for cleanly printing tables
import re
import math

def affiche(df, align="left", na_color="\033[91;3m", theme="newspaper"):
    """
    Display a pandas DataFrame with formatted table borders and styling.
    
    Args:
        df: pandas DataFrame to display
        align: text alignment ("left", "center", "right")
        na_color: ANSI color code for NaN values
        theme: border theme ("newspaper")
    """
    
    # Error handling
    if df.shape[1] == 0 or df.shape[0] == 0:
        msg = "That table doesn't exist!"
        width = len(msg)
        top = f"╔{'═' * (width + 2)}╗"
        mid = f"║ {msg} ║"
        bot = f"╚{'═' * (width + 2)}╝"
        print(f"{top}\n{mid}\n{bot}")
        return df
    
    # Theme setup
    if theme == "newspaper":
        border = {
            "h": "═", "v": "║",
            "tl": "╔", "tr": "╗",
            "bl": "╚", "br": "╝",
            "jn": "╬",
            "l": "╠", "r": "╣",
            "t": "╦", "b": "╩"
        }
    else:
        raise ValueError("Theme not supported. Try 'newspaper'")
    
    # ANSI helpers
    reset = "\033[0m"
    
    def color_na(x):
        if pd.isna(x):
            # Timestamp missing
            if isinstance(x, pd.Timestamp):
                return f"{na_color}NaT{reset}"
            # Everything else (int, bool, str, object, categorical) → NA
            return f"{na_color}NA{reset}"
        return str(x)


    # Width calculator (ignores ANSI codes)
    def display_width(s):
        clean = re.sub(r'\033\[[0-9;]*[mK]', '', str(s))
        return len(clean)
    
    # Data prep
    df_display = df.copy()
    for col in df_display.columns:
        df_display[col] = df_display[col].apply(
            lambda x: color_na("NaN") if pd.isna(x) else str(x)
        )
    
    col_names = list(df_display.columns)
    
    # Column widths
    col_widths = []
    for i, col in enumerate(col_names):
        header_width = display_width(col_names[i])
        data_widths = [display_width(val) for val in df_display.iloc[:, i]]
        col_widths.append(max([header_width] + data_widths))
    
    # Border drawing
    def draw_hline(connector_left, connector_right, cross):
        line = connector_left
        for i, width in enumerate(col_widths):
            line += border["h"] * (width + 2) + cross
        # Replace last cross with right connector
        line = line[:-1] + connector_right
        return line
    
    top_line = draw_hline(border["tl"], border["tr"], border["t"])
    mid_line = draw_hline(border["l"], border["r"], border["jn"])
    bot_line = draw_hline(border["bl"], border["br"], border["b"])
    
    # Header row
    header_parts = [border["v"]]
    for i, name in enumerate(col_names):
        width = col_widths[i]
        pad_total = width - display_width(name)
        
        if align == "left":
            pad_left = 0
        elif align == "center":
            pad_left = math.floor(pad_total / 2)
        elif align == "right":
            pad_left = pad_total
        else:
            pad_left = 0
        
        pad_right = pad_total - pad_left
        formatted_cell = f" {' ' * pad_left}{name}{' ' * pad_right} {border['v']}"
        header_parts.append(formatted_cell)
    
    header = "".join(header_parts)
    
    # Data rows
    data_rows = []
    for row_idx in range(len(df_display)):
        row_parts = [border["v"]]
        for col_idx in range(len(col_names)):
            content = df_display.iloc[row_idx, col_idx]
            width = col_widths[col_idx]
            pad_total = width - display_width(content)
            
            if align == "left":
                pad_left = 0
            elif align == "center":
                pad_left = math.floor(pad_total / 2)
            elif align == "right":
                pad_left = pad_total
            else:
                pad_left = 0
            
            pad_right = pad_total - pad_left
            formatted_cell = f" {' ' * pad_left}{content}{' ' * pad_right} {border['v']}"
            row_parts.append(formatted_cell)
        
        data_rows.append("".join(row_parts))
    
    # Final assembly
    print(top_line)
    print(header)
    print(mid_line)
    print("\n".join(data_rows))
    print(bot_line)

In [ ]:
# Define a custom method affiche() for cleanly printing tables
import re
import math

def affiche(self, align="left", na_color="\033[91;3m", theme="newspaper"):
    """
    Display a pandas DataFrame or Series with formatted table borders and styling.
    
    Args:
        self: the DataFrame or Series instance
        align: text alignment ("left", "center", "right")
        na_color: ANSI color code for missing values
        theme: border theme ("newspaper")
    
    Usage:
        df.affiche()
        df["column"].affiche()
    """
    
    df = self

    # Convert Series to DataFrame
    if isinstance(df, pd.Series):
        df = df.reset_index()

    # Handle empty DataFrame
    if df.shape[1] == 0 or df.shape[0] == 0:
        msg = "That table doesn't exist!"
        width = len(msg)
        top = f"╔{'═' * (width + 2)}╗"
        mid = f"║ {msg} ║"
        bot = f"╚{'═' * (width + 2)}╝"
        print(f"{top}\n{mid}\n{bot}")
        return df

    # Border theme
    if theme == "newspaper":
        border = {
            "h": "═", "v": "║",
            "tl": "╔", "tr": "╗",
            "bl": "╚", "br": "╝",
            "jn": "╬",
            "l": "╠", "r": "╣",
            "t": "╦", "b": "╩"
        }
    else:
        raise ValueError("Theme not supported. Try 'newspaper'")

    reset = "\033[0m"

    # Handle color for unique NA types
    def color_na(x):
        if pd.isna(x):
            # Timestamp missing
            if isinstance(x, pd.Timestamp):
                return f"{na_color}NaT{reset}"
            # Everything else (int, bool, str, object, categorical) → NA
            return f"{na_color}NA{reset}"
        return str(x)


    # Width calculator (ignores ANSI codes)
    def display_width(s):
        clean = re.sub(r'\033\[[0-9;]*[mK]', '', str(s))
        return len(clean)

    # Prepare display DataFrame
    df_display = df.copy()
    for col in df_display.columns:
        df_display[col] = df_display[col].apply(color_na)

    col_names = list(df_display.columns)

    # Column widths
    col_widths = []
    for i, col in enumerate(col_names):
        header_width = display_width(col)
        data_widths = [display_width(val) for val in df_display.iloc[:, i]]
        col_widths.append(max([header_width] + data_widths))

    # Draw horizontal line
    def draw_hline(connector_left, connector_right, cross):
        line = connector_left
        for i, width in enumerate(col_widths):
            line += border["h"] * (width + 2) + cross
        line = line[:-1] + connector_right
        return line

    top_line = draw_hline(border["tl"], border["tr"], border["t"])
    mid_line = draw_hline(border["l"], border["r"], border["jn"])
    bot_line = draw_hline(border["bl"], border["br"], border["b"])

    # Header
    header_parts = [border["v"]]
    for i, name in enumerate(col_names):
        width = col_widths[i]
        pad_total = width - display_width(name)

        if align == "left":
            pad_left = 0
        elif align == "center":
            pad_left = math.floor(pad_total / 2)
        elif align == "right":
            pad_left = pad_total
        else:
            pad_left = 0

        pad_right = pad_total - pad_left
        header_parts.append(f" {' ' * pad_left}{name}{' ' * pad_right} {border['v']}")
    header = "".join(header_parts)

    # Data rows
    data_rows = []
    for row_idx in range(len(df_display)):
        row_parts = [border["v"]]
        for col_idx in range(len(col_names)):
            content = df_display.iloc[row_idx, col_idx]
            width = col_widths[col_idx]
            pad_total = width - display_width(content)

            if align == "left":
                pad_left = 0
            elif align == "center":
                pad_left = math.floor(pad_total / 2)
            elif align == "right":
                pad_left = pad_total
            else:
                pad_left = 0

            pad_right = pad_total - pad_left
            row_parts.append(f" {' ' * pad_left}{content}{' ' * pad_right} {border['v']}")
        data_rows.append("".join(row_parts))

    # Print table
    print(top_line)
    print(header)
    print(mid_line)
    print("\n".join(data_rows))
    print(bot_line)

    return None

# Monkey-patch
pd.DataFrame.affiche = affiche
pd.Series.affiche = affiche


In [ ]:
# Define a custom function count_table() for generating value counts and percentages
import numpy as np

def count_table(df, column):
    """
    Usage: count_table(df, "column") 
    """
    # Get value counts and convert to DataFrame
    table = df[column].value_counts(dropna = False).reset_index()
    table.columns = [column, "count"]
    
    # Calculate percentages
    table["percent"] = (table["count"] / table["count"].sum() * 100).round(0).astype(int).astype(str) + "%"
    
    # Handle the <1% case
    table["percent"] = np.where(
        (table["percent"] == "0%") & (table["count"] != 0),
        "<1%",
        table["percent"]
    )
    
    return table

In [ ]:
# Define a custom method count_table() for generating value counts and percentages
import pandas as pd
import numpy as np

def count_table(self):
    """
    Create a count table with counts and percentages for a Series.
    Usage: series.count_table() or df["column"].count_table()
    """
    # Get value counts and convert to DataFrame
    table = self.value_counts(dropna = False).reset_index()
    table.columns = [self.name or "value", "count"]
    
    # Calculate percentages
    table["percent"] = (table["count"] / table["count"].sum() * 100).round(0).astype(int).astype(str) + "%"
    
    # Handle the <1% case
    table["percent"] = np.where(
        (table["percent"] == "0%") & (table["count"] != 0),
        "<1%",
        table["percent"]
    )
    
    return table

# Monkey patch the method onto Series
pd.Series.count_table = count_table

In [ ]:
# Define a custom function count_na() for counting NA values for each column
def count_na(df: pd.DataFrame) -> pd.DataFrame:
    """
    Usage: count_na(df) 
    """
    # Count missing values per column
    na_counts = df.isna().sum()
    
    # Build result table
    result = pd.DataFrame({
        "col": na_counts.index,
        "na_count": na_counts.values
    })
    
    # Add percentage labels 
    result["na_percent"] = result["na_count"].apply(
        lambda x: (
            "0%" if x == 0 
            else "<1%" if x / len(df) <= 0.0099 
            else f"{round(x / len(df) * 100):.0f}%"
        )
    )
    
    # Sort by missing count
    result = result.sort_values(by="na_count", ascending=False).reset_index(drop=True)
    
    return result

In [ ]:
# Define a custom method count_na() for counting NA values for each column
def count_na(self):
    """
    Count NA values for each column in a DataFrame, with percentages.
    Usage: df.count_na()
    """
    # Count NAs for each column
    na_counts = self.isna().sum().reset_index()
    na_counts.columns = ["col", "na_count"]

    # Calculate percentages
    na_counts["na_percent"] = (
        (na_counts["na_count"] / len(self) * 100)
        .round(0)
        .astype(int)
        .astype(str) + "%"
    )

    # Handle 0% and <1% cases
    na_counts["na_percent"] = np.where(
        na_counts["na_count"] == 0,
        "0%",
        np.where(
            (na_counts["na_count"] / len(self)) <= 0.0099,
            "<1%",
            na_counts["na_percent"]
        )
    )

    # Sort by missing count
    na_counts = na_counts.sort_values("na_count", ascending=False).reset_index(drop=True)
    return na_counts

# Monkey patch the method onto DataFrame
pd.DataFrame.count_na = count_na

# 1. Simple Data

In [ ]:
# We use print for displaying something
print("Hello world!")

In [ ]:
# Our assignment operator is the equals sign (=)
x = 10
y = 20

print(x)
print(y)

In [ ]:
# We roughly use the same mathematical operators
z = x * y
print(z)

a = x / y
print(a)

b = x - y
print(b)

In [ ]:
# Let's look at some base Python data types
# NB: Pandas and Numpy introduce data types not present in base Python
k = 1000
name = "Henry"
does_exist = True

# In Python, it prefers we specify print()
print(type(k))
print(type(name))
print(type(does_exist))

In [ ]:
# Missing values operate differently as objects vs as parts of a series
# Note: pd.NA is the NA placeholder for character strings
none_obj = None
na_series = pd.Series(["apple", pd.NA, 25])

print(none_obj)
print(na_series)

# 2. Working with Dataframes

In [ ]:
# Create a dataframe
# Note: We use np.nan for numeric NAs
df = pd.DataFrame({
    "Name": ["Henry", "Bob", "Joanne", "Steven"], 
    "Age": [22, np.nan, 30, 48],
    "Birthday": ["2003-12-29", "1980-05-15", "1995-01-12", pd.NA]
})

print(df)

In [ ]:
# Display info on the dataframe
df.info()

In [ ]:
# Access a single column by name
df["Birthday"]

In [ ]:
# Access the first column by position
# Note: Python starts counting at 0
# Note: _.iloc() uses integer-location based indexing
column_1 = df.iloc[:, 0]

print(column_1)

In [ ]:
# Add a new column
df["Fav_Animal"] = ["Cat", "Penguin", "Sloth", "Dog"]

print(df)

In [ ]:
# Drop a column
df.drop("Age", axis = 1, inplace = True)

print(df)

In [ ]:
# Convert birthday to a datefield with pd.to_datetime()
df["Birthday"] = pd.to_datetime(df["Birthday"])

df["Birthday"].info()

In [ ]:
# NB: Re-defining an object creates a *reference* to that object
reference = df

# This means that modifying the original object will modify the new one
df.drop("Name", axis = 1, inplace = True)

print(reference)

In [ ]:
# In Python, we have to use the copy() method to create a true copy like R
copy = df.copy()

# Now, when we modify the original, the copy won't be modified
df.drop("Birthday", axis = 1, inplace = True)

print(copy)

# 3. More Advanced Manipulation

#### 1. Reading in data

In [ ]:
# Read in the calls for service data
# Note: In Python, certain functions must have their packages named 
cfs = pd.read_csv("./data/calls_for_service_2025_demo.csv")

# Preview the first 10 rows
# Note: Here, _.head() is a method (ie, a function specific to a given type of object)
print(cfs.head(10))

In [ ]:
# Use our custom affiche() method for better readability
# Note: We granted {pandas} dataframes a new method affiche() for printing tables
cfs.head(10).affiche()

#### 2. Cleaning column names

In [ ]:
# Apply the clean_names() method from {pyjanitor}
# Note, {pyjanitor} grants {pandas} dataframes a new method for cleaning names
cfs = cfs.clean_names()

cfs.info()

In [ ]:
# We can select data with select() from {siuba}
# Note: We use the backslash ( \ ) to extend a statement to the next line
# Note: siuba uses the underscore ( _ ) as a placeholder for the column's dataframe
# And we'll use reset_index(), similar to R's ungroup() from {dplyr}
cfs >> select(_.nopd_item) \
    >> head(10) \
    >> _.reset_index() \
    >> _.affiche()

# Note: This is the same as:
# cfs["nopd_item"].head(10).affiche()

In [ ]:
# Rename columns (in case they weren't already renamed)
# We'll use rename() from {siuba} 
# Where rename wants "new = _.old"
cfs = cfs >> rename(zip_code = _.zip)

# Check on our 19th column (note: again, Python starts counting at 0)
cfs.columns[18]

In [ ]:
# We're actually going to do some more renaming to match R's {janitor}
cfs = cfs >> rename(
    type_text = _.typetext,
    initial_type = _.initialtype,
    initial_type_text = _.initialtypetext,
    initial_priority = _.initialpriority,
    map_x = _.mapx,
    map_y = _.mapy,
    time_create = _.timecreate, 
    time_dispatch = _.timedispatch,
    time_arrive = _.timearrive,
    time_closed = _.timeclosed,
    disposition_text = _.dispositiontext,
    self_initiated = _.selfinitiated,
    police_district = _.policedistrict
)

# Let's see if our column names match now
cfs.info()

# Note: Looks like there are some differences in data types
# We'll try to get these to line up later

#### 3. Converting character fields to sentence case

In [ ]:
# Acquire character (object) columns with the select_dtypes() method
str_cols = cfs.select_dtypes(include = "object").columns  

# Iterate through each column with str.capitalize() from {pandas}
for col in str_cols:
    cfs[col] = cfs[col].str.capitalize()

In [ ]:
# Now we can look at the dispositions and see what they look like
# Dataframes have a method called value_counts() for grouping and counting values
cfs["disposition_text"].value_counts().affiche()

#### 4. Re-casting data types

In [ ]:
# Let's investigate map_x:
# From now on, we'll use our custom count_table() method
cfs["zip_code"].info()
cfs["zip_code"].count_table().affiche()

In [ ]:
# Looks like it's a float in Python, likely because it coerced the "None" value
# If we had to, we could use the astype() method to recast it as a float
cfs = cfs >> mutate(
    zip_code = _.zip_code.astype(float))

# Now let's look at zip_code
cfs["zip_code"].info()
cfs["zip_code"].count_table().affiche()

In [ ]:
# Let's go back and look at the rest of our data types
cfs.info()

In [ ]:
# Let's investigate the time_create column 
cfs["time_create"].count_table().affiche()

In [ ]:
# Let's recast this field and the other date fields with pd.to_datetime
time_cols = ["time_create", "time_dispatch", "time_arrive", "time_closed"]

for col in time_cols:
    cfs[col] = pd.to_datetime(cfs[col], errors = "raise", format = "mixed")

# Now let's select() the date columns and see their data types
# Note: We can also use the underscore placeholder _ from {siuba} here on info()
# Note: We'll use Python's unpacking operator * for passing a list to select()
cfs >> select(*time_cols) >> _.info()

In [ ]:
# Let's go back to our columns
# And all of our data types look pretty good
cfs.info()

#### 5. Replacing missing values

##### A. Replacing 0s

In [ ]:
# With some digging, we can find that some columns have 0s 
# They're map_x, map_y, and police_district 
# Here's police district 
cfs["police_district"].count_table().affiche()

In [ ]:
# Replace 0s in map_x, map_y, and police_district 
zero_cols = ["map_x", "map_y", "police_district"]

for col in zero_cols:
    cfs[col] = cfs[col].replace(0, pd.NA)

# Now, let's see what police districts look like
cfs["police_district"].count_table().affiche()

##### B. Replacing Nones

In [ ]:
# While there are no "None" values in the {pandas} version of the cfs table ...
# this is how we might find them
cfs["beat"].count_table() \
    >> filter(_.beat == "None") \
    >> _.affiche()

In [ ]:
# We would use a simpler approach to the one we used with replacing 0s
cfs["beat"] = cfs["beat"].replace("None", pd.NA)

# Now we would see if they were succesfully transormed to NAs
cfs["beat"].count_table() \
    >> filter(_.beat.isna()) \
    >> _.affiche()

In [ ]:
# Now we can look at our missing values by column:
cfs.count_na().affiche()

#### 6. Using regex to update a column

In [ ]:
# Next, let's take a look at other unexpected beat values
# We're looking for strings that don't follow the conventional pattern of ...
# ... 1 digit followed by 1 letter, then 2 more digits (eg 1b01)
# This is a common issue resulting from programs like Excel Excel auto-parsing certain fields
# Basically, we want 1.00e+03 to become 1e03, etc 
# We'll create a new column with an if_else() requiring a condition and arguments for true or false
cfs = cfs >> mutate(
    beat_2 = if_else(
        (~_.beat.astype("string").str.contains(r"\d{1}[a-z]\d{2}")) & (_.beat.notna()),
        true = _.beat.astype("string").str.replace(pat = r"(\d{1})\.00(e)\+(\d{2})", 
                                                   repl = r"\1\2\3", 
                                                   regex = True),
        false = _.beat
        )
    )

cfs \
    >> filter(~_.beat.astype("string").str.contains(r"\d{1}[a-z]\d{2}") & _.beat.notna()) \
    >> distinct(_.beat, _.beat_2) \
    >> _.affiche()

In [ ]:
# Now we can update beat with our beat_2 column
cfs["beat"] = cfs["beat_2"]

# Then drop the beat_2 column
cfs.drop("beat_2", axis = 1, inplace = True)

# And look at the new beat values that were updated
cfs["beat"] \
    >> _.count_table() \
    >> filter(_.beat.astype("string").str.contains(r"e")) \
    >> _.affiche()

#### 7. Creating booleans

In [ ]:
# It looks like self-initiated uses a Y / N system 
cfs["self_initiated"].count_table().affiche()

In [ ]:
# So we're going to make it a boolean instead with another if_else() 
cfs = cfs >> mutate(self_initiated = if_else(
    _.self_initiated == "Y",
    true = True,
    false = False)
)

In [ ]:
# Let's see how that turned out
cfs["self_initiated"].info()

cfs["self_initiated"].count_table().affiche()

#### 8. Extracting coordinate info

In [ ]:
# Let's take a look at the location field
cfs["location"].info()

cfs["location"].head().affiche()

In [ ]:
# Let's extract the longitude field from the location column
# And coerce as float type

# Key: \- is an escaped search for a hyphen 
# \d{2} is an escaped search for two digits
# \. is an escaped search for a decimal
# and \d{4,} is an escaped search for four or more digits
longitude_pattern = r"(\-\d{2}\.\d{4,})"
cfs["longitude"] = cfs["location"].str.extract(longitude_pattern).astype(float)

cfs["longitude"].head().affiche()

In [ ]:
# Let's extract the latitude field from the location column
# And coerce as float type

# Key: \s is an escaped search for a whitespace
# \d{2} is an escaped search for two digits
# \. is an escaped search for a decimla
# and \d{7,} is an escaped search for foiur or more digits
latitude_pattern = r"\s(\d{2}\.\d{4,})"

cfs["latitude"] = cfs["location"].str.extract(latitude_pattern).astype(float)

cfs["latitude"].head().affiche()

#### 9. Geocoding missing location data

In [ ]:
# Identify cases with missing location data
missing = cfs \
    >> filter(_.longitude.isna() & _.latitude.isna()) \
    >> filter(_.block_address.notna()) 
    
missing \
    >> select(_.block_address, _.zip_code, _["location":"latitude"]) \
    >> head(10) \
    >> _.affiche()
    
print(f"Number of missing location fields: {len(missing)}")

In [ ]:
# Let's fill out an address field
# Note: In Python, we convert the zip to an int, then a string 
missing["address"] = missing["block_address"] + ", Louisiana, LA " + \
    missing["zip_code"].astype("Int64").astype("string").where(missing["zip_code"].notna(), "")

missing \
    >> select(_.block_address, _.address) \
    >> head(10) \
    >> _.affiche()
    
# geocode() from {geopy}/{geopandas} automatically returns a GeoDataFrame
missing_gdf = geocode(missing["address"], provider = "arcgis", timeout = 10)
missing["lat"] = missing_gdf.geometry.y
missing["long"] = missing_gdf.geometry.x

In [ ]:
# First let's fix the new latitude and longitude columns
# Since Python kept our original columns, we'll update them with the new columns
missing["longitude"] = missing["long"]
missing["latitude"] = missing["lat"]

# Now let's look at the geocoded addresses
missing \
    >> select(_.address, _.latitude, _.longitude) \
    >> head(10) \
    >> _.affiche()

In [ ]:
# Let's double-check if there are any missing coordinates
missing \
    >> filter(_.longitude.isna() | _.latitude.isna()) \
    >> head(10) \
    >> _.affiche()

In [ ]:
# Now let's join them back to the original cfs table
cfs = cfs \
    >> left_join(_, (missing >> select(_.block_address, 
                                       _.latitude, 
                                       _.longitude)), 
                 by = "block_address")
    
# We'll update the original longitude and latitude values with the new ones via fillna(), ...
# ... drop the extra columns, and rename the original columns
cfs = cfs \
    >> mutate(longitude_x = _.longitude_x.fillna(_.longitude_y), 
              latitude_x = _.latitude_x.fillna(_.latitude_y)) \
    >> select(~_["longitude_y", "latitude_y"]) \
    >> rename(longitude = _.longitude_x, 
              latitude = _.latitude_x)
    
cfs.info()

#### 10. Calculating response time

In [ ]:
# Calculate response time (from when call was created to when the officer arrived)
cfs = cfs \
    >> mutate(response_time = _.time_arrive - _.time_create)

# We'll have to reorder the columns manually in Python
cfs = cfs >> select(
    _["nopd_item":"time_arrive"], 
    _.response_time,
    _["time_closed":"latitude"]
)

# We see some zeroes here — let's make them NA
cfs["response_time"] = cfs["response_time"].replace("0", pd.NaT)

# Look at the longest response times
cfs \
    >> select(_["time_create":"response_time"]) \
    >> filter(_.time_arrive.notnull()) \
    >> arrange(-_.response_time) \
    >> head(10) \
    >> _.affiche()

In [ ]:
# Let's look at that case with the longest response time
cfs \
    >> filter(_.response_time.astype("string").str.contains("1 days")) \
    >> _.affiche()

#### 11. Collapsing call priorities

In [ ]:
# We'll collapse the call priorities using the descriptions given from the codebook
cfs = cfs >> mutate(
    priority_desc = case_when({
            _.priority.str.contains("0"): "Non-police",
            _.priority.str.contains("1"): "Non-emergency",
            _.priority.str.contains("2"): "Emergency",
            _.priority.str.contains("3"): "Officer assistance"
        })
    )

# And in Python, we'll need to rearrange eveything again
cfs = cfs >> select(
    _["nopd_item":"priority"], 
    _.priority_desc,
    _["initial_type":"latitude"]
)

In [ ]:
# Take a look at the new priority descriptions
cfs["priority_desc"].count_table().affiche()

#### 12. Classifying call types

In [ ]:
# Define keywords for violent crimes
violent_keywords = ["assault", "battery", "homicide", "fight", "rape", "carjacking"]

# Define keywords for theft crimes
theft_keywords = ["theft", "burglary", "stolen", "shoplifting", "damage"]

# Define keywords for traffic incidents
traffic_keywords = ["traffic", "driving", "tow", "accident", "stranded"]

# Categorize calls
cfs = cfs >> mutate(
    is_violent = _.type_text.str.contains("|".join(violent_keywords), regex = True, case = False),
    is_theft = _.type_text.str.contains("|".join(theft_keywords), regex = True, case = False),
    is_traffic = _.type_text.str.contains("|".join(traffic_keywords), regex = True, case = False), 
    category = case_when({
        _.is_violent == True: "Violent",
        _.is_theft == True: "Theft",
        _.is_traffic == True: "Traffic", 
        True: "Other"})
  ) >> select(~_["is_violent":"is_traffic"])

# And in Python, we'll need to rearrange eveything again
cfs = cfs >> select(
    _["nopd_item":"type_text"], 
    _.category,
    _["priority":"latitude"]
)

# 4. Exploratory Data Analysis

#### 1. Summary statistics

In [ ]:
# Use the describe() method from {pandas}
cfs \
    >> select(~_.startswith("time")) \
    >> _.describe() \
    >> _.reset_index() \
    >> _.affiche()

#### 2. What are the most commonly occuring call type categories?

In [ ]:
cfs["category"].count_table().affiche()

#### 3. When do most calls take place?

In [ ]:
# Extract the hour from the time_create field
# We'll also calculate a time period of day for later yse
cfs = cfs >> mutate(
    hour = _.time_create.dt.hour,
    time_period = case_when({
        ((_.hour >= 0) & (_.hour <= 5)): "Night",
        ((_.hour >= 6) & (_.hour <= 11)): "Morning",
        ((_.hour >= 12) & (_.hour <= 17)): "Afternoon",
        ((_.hour >= 18) & (_.hour <= 23)): "Evening"
    })
)

# And in Python, of course we need to re-arrange
cfs = cfs >> select(
    _["nopd_item":"time_create"], 
    _.hour, _.time_period,
    _["time_dispatch":"latitude"]
)

hourly_sum = cfs["hour"].count_table() \
    >> arrange(_.hour)

# We can use ggplot() from {plotnine} + geom_line()
ggplot(hourly_sum, aes(x = "hour", y = "count")) + \
    geom_line()

In [ ]:
# Let's look at those time periods too
cfs["time_period"].count_table().affiche()

#### 4. What are the most common call dispositions?

In [ ]:
cfs["disposition_text"].count_table().affiche()

#### 5. Which police districts receive the most calls?

In [ ]:
cfs["police_district"].count_table().affiche()

#### 6. What proportion of calls are self-initiated?

In [ ]:
cfs["self_initiated"].count_table().affiche()

#### 7. Where do most calls originate from?

In [ ]:
# Read shapefile and filter to Orleans Parish
# As well as filter out water-only polygons
shapes = gp.read_file("./data/shapes/tl_2024_22_bg.shp") \
    >> filter(_.COUNTYFP == "071") \
    >> filter(~_.GEOID.astype("string").str.contains("220719900000|220719801001", regex = True))
    
# Preview the orleans shapes map
ggplot(shapes, aes()) + \
  geom_map(fill = "lightgrey", color = "grey") + \
  labs(title = "New Orleans shapefile")

In [ ]:
# Convert the cfs object to a geopandas (gp) object
cfs_gp = gp.GeoDataFrame(cfs, 
                         geometry = gp.points_from_xy(cfs.longitude, cfs.latitude), 
                         crs = "EPSG:4326")

# Look at the shapes map with cfs overlaid
ggplot() + \
  geom_map(data = shapes, fill = "lightgrey", color = "grey") + \
  geom_map(data = cfs_gp, alpha = .5, size = .75) + \
  labs(title = "New Orleans calls for service") 

#### 8. What is the best predictor of response time?

In [ ]:
# Note: I only included variables that were statistically significant for readability
# We'll also exclude all self-initiated calls if present
# As well as convert response time to a true numeric column (in seconds)
# And drop NAs
data = cfs \
    >> _.copy() \
    >> filter(~_.self_initiated) \
    >> select(_.category, _.priority_desc, _.time_period, 
              _.response_time, _.police_district) \
    >> mutate(response_time = _.response_time.dt.seconds) \
    >> _.dropna()
    
# We'll also convert our object columns to categories
obj_cols = ["category", "priority_desc", "time_period", "police_district"]

for col in obj_cols:
    data[col] = data[col].astype("category")
         
data.info()

In [ ]:
# Fit a multiple linear regression model
model = smf.ols(
    formula = "response_time ~ category + priority_desc + \
    time_period  + police_district", 
    data = data)

# TO-DO: Invesitage why model outputs differ
print(model.fit().summary())